# Joint Search Viewer
View the results of joint pose search

In [7]:
from pathlib import Path
import sys
import os
import random
import torch
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

root_dir = Path().resolve().parent
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

from utils import util
from joint.joint_environment import JointEnvironment
from joint.joint_prediction_set import JointPredictionSet

from search.search_simplex import SearchSimplex
from search.search_random import SearchRandom

from train import JointPrediction
from datasets.joint_graph_dataset import JointGraphDataset

## Load Network & Data
Load a pretrained checkpoint to use for inference.

Load the dataset and create an instance of the JointPredictionSet class.
We assume that the joint json and mesh part files are in the same directory.

In [8]:
def load_network(checkpoint_file):
    """Load the network"""
    if not checkpoint_file.exists():
        print("Checkpoint file does not exist")
        return None
    model = JointPrediction.load_from_checkpoint(
        checkpoint_file,
        map_location=torch.device("cpu")  # Just use the CPU
    )
    return model

# Use a checkpoint from the pretrained model
checkpoint_file = root_dir / "pretrained/paper/last_run_0.ckpt"
model = load_network(checkpoint_file)

# Change to point to the Fusion 360 Gallery joint dataset
# this directory should contain the joint json and obj part files
data_dir = root_dir / "data/tester"
dataset = JointGraphDataset(
    root_dir=data_dir,
    split="val",
    label_scheme="Joint,JointEquivalent"
)
# Data sample in the dataset we want to visualize
index = 2
# Graphs for part one and two, and the densely connected joint graph
g1, g2, joint_graph = dataset[index]
# The joint file json
joint_file = data_dir / dataset.files[index]
# Random seed to use
seed = 24

# Create the prediction with the given data and model
jps = JointPredictionSet(
    joint_file,
    g1, g2, joint_graph,
    model,
    seed=seed
)

Data cache loaded from: /Users/willisk/Autodesk/Code/Github/JoinABLe/data/tester/val.pickle


## Ground Truth
View the ground truth assembled state of the parts.

In [9]:
v, f, c, e = jps.get_meshes(
    joint_index=0,
    show_joint_entity_colors=False,
    show_joint_equivalent_colors=False,
)
p = mp.plot(v, f, c=c);

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(18.776182…

## Joint Axis Prediction without Search
View the parts when assembled using only the joint axis prediction, without performing search for the offset/rotation/flip parameters.

In [10]:
# Get the transform to move body1, to align with the static body2
# using default parameters, i.e. no offset, rotation, or flip
transform = JointEnvironment.get_transform_from_parameters(
    jps,
    prediction_index=0,  # Top-1 prediction
    offset=0,
    rotation_in_degrees=0,
    flip=False
)

# Render the meshes to visualize
v, f, c, e, n, ni = jps.get_meshes(
    apply_transform=True,
    body_one_transform=transform,
    show_joint_entity_colors=False,
    show_joint_equivalent_colors=False,
    return_vertex_normals=True
)
p = mp.plot(v, f, c=c);

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(22.458543…

## Joint Pose Search
Perform joint pose search to find the offset, rotation, and flip parameters.

In [11]:
random_state = np.random.RandomState(seed)
# Nelder–Mead Simplex Search as used in the paper
search = SearchSimplex(random_state=random_state)
# Random search can also be used
# search = SearchRandom(random_state=random_state, budget=500)

result = search.search(jps)
# Returns a dict with:
# - prediction_index: Index of the prediction from the network, 0 being the highest probability
# - offset: Offset parameter
# - rotation: Rotation parameter
# - flip: Flip parameter
# - transform: Transform created from the axis and parameters, apply this transform to body 2 will assemble the parts together
# - evaluation: Evaluation score where lower is better
# - overlap: Overlap between the parts
# - contact: Contact between the parts
result

{'prediction_index': 37,
 'offset': 2.615928649902344e-06,
 'rotation': 0.00046525239944457977,
 'flip': True,
 'transform': array([[ 0.76620711,  0.        ,  0.6425937 ,  3.90841466],
        [ 0.        , -1.        ,  0.        ,  0.00784779],
        [-0.6425937 ,  0.        ,  0.76620711,  6.01150962],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 'evaluation': -1.102427577096359,
 'overlap': 0.0021510395545753275,
 'contact': 0.11045786166509343}

In [12]:
# Visualized the 
v, f, c, e, n, ni = jps.get_meshes(
    apply_transform=True,
    body_one_transform=result["transform"],
    show_joint_entity_colors=False,
    show_joint_equivalent_colors=False,
    return_vertex_normals=True
)
p = mp.plot(v, f, c=c);

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(9.9963521…